In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from pathlib import Path
import logging
import random
import textwrap
import itertools
from tqdm.auto import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from scipy.spatial.distance import cdist

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel

from sentiment_analysis.config import SentimentAnalysisConfig
from sentiment_analysis.data_access import DataClass
from sentiment_analysis.features import Features
from sentiment_analysis.evaluation import CustomEvaluation
from sentiment_analysis.utils.constants import (
    TEXT,
    TARGET,
    ORIGINAL_TEXT,
    SPLIT,
    TRAIN,
    VALID,
    TEST,
    SAVED_MODELS,
    PREDICTION
)

device = "cuda" if torch.cuda.is_available() else "cpu"
sns.set_style("darkgrid")
PARENT_PATH = Path(os.getcwd()).parent.absolute()
FORMAT = "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
logging.basicConfig(format=FORMAT, level=logging.INFO)

seed = 128
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed);

In [ ]:
config = SentimentAnalysisConfig()
config.CURRENT_PATH = PARENT_PATH

data = DataClass(config)
df = data.build()
df.head(10)

In [ ]:
df_features.head(3)

Prepare data for training and validation

In [ ]:
train = df_features[df_features[SPLIT].isin([TRAIN])].copy()
valid = df_features[df_features[SPLIT].isin([VALID])].copy()
train.reset_index(drop=True, inplace=True)
valid.reset_index(drop=True, inplace=True)

X_train = train[features.vectorizer.get_feature_names_out()].copy()
X_valid = valid[features.vectorizer.get_feature_names_out()].copy()
Y_train = train[TARGET]
Y_valid = valid[TARGET]

pos_prob_train = sum(Y_train) / len(Y_train)
pos_prob_valid = sum(Y_train) / len(Y_train)
print(f"Number of features: {X_train.shape[1]:,}")
print(f"Number of training samples: {X_train.shape[0]:,}")
print(f"Training set label distribution: pos:{pos_prob_train:0.2f}, neg:{1-pos_prob_train:0.2f}")
print(f"Number of validation samples: {X_valid.shape[0]:,}")
print(f"Validation set label distribution: pos:{pos_prob_valid:0.2f}, neg:{1-pos_prob_valid:0.2f}")

Define custom dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(
        self,
        data: pd.DataFrame,
        model_name: str,
        max_length: int = 128
    ):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.data = data
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        row = self.data.iloc[index]
        text = row[ORIGINAL_TEXT]
        target = np.array(row[TARGET]).reshape(-1,)
        inputs = self.tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=self.max_length,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        return_token_type_ids=True,
        return_tensors='pt'
        )

        return {
            "text": text,
            "input_ids": inputs["input_ids"].flatten(),
            "attention_mask": inputs["attention_mask"].flatten(),
            "token_type_ids": inputs["token_type_ids"].flatten(),
            "target": torch.FloatTensor(target)
        }

Define the pre-trained model with classifier head

In [ ]:
class Model(nn.Module):
    def __init__(self, model_name: str):
        super().__init__()
        self.l1 = AutoModel.from_pretrained(model_name)
        self.l2 = nn.Dropout(0.3)
        self.l3 = nn.Linear(self.l1.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        out = self.l1(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        out = self.l2(out.pooler_output)
        out = self.l3(out)
        return out

Choose model and load the tokenizer

In [ ]:
MODEL_NAME = 'roberta-base'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Token-length analysis based on tokenizer

This is important in deciding `MAX_TOKEN_COUNT`

In [ ]:
token_counts = []
for _, row in train.iterrows():
    tokens = tokenizer.encode(
        row[ORIGINAL_TEXT],
        max_length=1000,
        truncation=True
    )
    token_counts.append(len(tokens))
sns.kdeplot(token_counts)
plt.xlabel("# Tokens")
plt.ylabel("Density")
plt.show();

In [ ]:
Initialize model with pre-trained weights

In [ ]:
model = Model(MODEL_NAME)
model.to(device);

Define hyper-parameters, choose criterion, optimizer

In [ ]:
MAX_TOKEN_COUNT = 256
TRAIN_BATCH = 32
VALID_BATCH_SIZE = 32
EPOCHS = 5
LEARNING_RATE = 1e-05
NUM_WORKERS = 0

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)
saved_models_path = Path(os.path.join(data.current_path, SAVED_MODELS))
saved_models_path.mkdir(parents=True, exist_ok=True)

Build dataloaders

In [ ]:
train_dataset = CustomDataset(
    train,
    MODEL_NAME,
    max_length=MAX_TOKEN_COUNT
)
valid_dataset = CustomDataset(
    valid,
    MODEL_NAME,
    max_length=MAX_TOKEN_COUNT
)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=TRAIN_BATCH, num_workers=NUM_WORKERS)
valid_loader = DataLoader(valid_dataset, shuffle=False, batch_size=VALID_BATCH_SIZE, num_workers=NUM_WORKERS)

Train

In [ ]:
# Move model to device
model.to(device)
# Lists to keep the losses and evaluation scores at the end of each epoch
train_loss, valid_loss = list(), list()
for epoch in range(EPOCHS):
    # Dummy lists to keep the losses and evaluation scores at the end of each iteration
    # (one batch forward and backward process)
    train_batch_loss, valid_batch_loss = list(), list()
    # Training mode
    model.train()
    for i, item in enumerate(train_loader):
        # Move input to device
        input_ids = item["input_ids"].to(device, dtype=torch.long)
        attention_mask = item["attention_mask"].to(device, dtype=torch.long)
        token_type_ids = item["token_type_ids"].to(device, dtype=torch.long)
        target = item["target"].to(device, dtype=torch.float)
        # Forward pass
        out = model(input_ids, attention_mask, token_type_ids)
        # Calculate loss
        loss = criterion(out, target)
        # Reset gradients
        optimizer.zero_grad()
        # Backward pass (backpropogation)
        loss.backward()
        # Update weights
        optimizer.step()
        # Collect loss and evaluation scores
        train_batch_loss.append(loss.item())
        message = \
            f"EPOCH:{epoch+1}/{EPOCHS}, " + \
            f"step:{i+1}/{len(train_loader)}, " + \
            f"loss={loss.item()}"
        print("\r", message, end="")
    # Take the average of iteration losses and evaluation scores
    # and append it to the epoch losses list
    train_loss.append(np.array(train_batch_loss).mean())
    # Evaluation mode
    model.eval()
    with torch.no_grad():
        for i, item in enumerate(valid_loader):
            # Move input to device
            input_ids = item["input_ids"].to(device, dtype=torch.long)
            attention_mask = item["attention_mask"].to(device, dtype=torch.long)
            token_type_ids = item["token_type_ids"].to(device, dtype=torch.long)
            target = item["target"].to(device, dtype=torch.float)
            # Forward pass
            out = model(input_ids, attention_mask, token_type_ids)
            # Calculate loss
            loss = criterion(out, target)
            valid_batch_loss.append(loss.item())
            message = \
                f"EPOCH:{epoch+1}/{EPOCHS}, " + \
                f"step:{i+1}/{len(valid_loader)}, " + \
                f"loss={loss.item()}"
            print("\r", message, end="")
    # Take the average of iteration losses and evaluation scores
    # and append it to the epoch losses list
    valid_loss.append(np.array(valid_batch_loss).mean())
    message = \
        f"EPOCH:{epoch+1}/{EPOCHS} - " + \
        f"Training Loss: {train_loss[-1]}, " + \
        f"Validation Loss: {valid_loss[-1]}"
    print("\r", message)
    # Save model
    state = f"epoch_{epoch+1:03}.pth"
    state_dict_path = os.path.join(saved_models_path, state)
    torch.save(model.state_dict(), state_dict_path)

In [ ]:
best_epoch = np.argmin(valid_loss) + 1
print(f"Best epoch: {best_epoch}")
state = f"epoch_{best_epoch:03}.pth"
state_dict_path = os.path.join(saved_models_path, state)
state_dict = torch.load(state_dict_path)
model.load_state_dict(state_dict)
model.to(device)
model.eval();

In [ ]:
fig = plt.figure(figsize=(8, 6))
plt.plot(range(1, EPOCHS+1), train_loss, label="Train Loss", color="orange")
plt.plot(range(1, EPOCHS+1), valid_loss, label="Validation Loss", color="blue")
plt.axvline(x=best_epoch, label="Best Epoch", color="darkgreen", linestyle="dashdot")
plt.xlabel("Epoch")
plt.ylabel(f"Loss")
plt.title(f"Best Epoch: {best_epoch}")
plt.legend(loc="upper right");

In [ ]:
Evaluation

In [ ]:
eval = CustomEvaluation()

In [ ]:
valid_dataset = CustomDataset(
    valid,
    MODEL_NAME,
    max_length=MAX_TOKEN_COUNT
)
valid_loader = DataLoader(valid_dataset, shuffle=False, batch_size=1, num_workers=NUM_WORKERS)

y_true = []
y_pred_probab = []
with torch.no_grad():
    for i, item in enumerate(valid_loader):
        input_ids = item["input_ids"].to(device, dtype=torch.long)
        attention_mask = item["attention_mask"].to(device, dtype=torch.long)
        token_type_ids = item["token_type_ids"].to(device, dtype=torch.long)
        target = item["target"].to(device, dtype=torch.float)

        out = model(input_ids, attention_mask, token_type_ids)
        out = torch.sigmoid(out)

        y_true.append(target.to("cpu").numpy().reshape(-1,))
        y_pred_probab.append(out.to("cpu").numpy().reshape(-1,))

y_true_val = np.array(y_true).astype(int)
y_pred_probab_val = np.array(y_pred_probab)

Score on validation set

In [ ]:
y_pred_val = np.where(y_pred_probab_val > .5, 1, 0)

eval.evaluate(y_true=y_true_val, y_pred=y_pred_val)

Score on test set

In [ ]:
test = df_features[df_features[SPLIT].isin([TEST])].copy()
test.reset_index(drop=True, inplace=True)

test_dataset = CustomDataset(
    test,
    MODEL_NAME,
    max_length=MAX_TOKEN_COUNT
)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=1, num_workers=NUM_WORKERS)

y_true = []
y_pred_probab = []

with torch.no_grad():
    for i, item in enumerate(test_loader):
        input_ids = item["input_ids"].to(device, dtype=torch.long)
        attention_mask = item["attention_mask"].to(device, dtype=torch.long)
        token_type_ids = item["token_type_ids"].to(device, dtype=torch.long)
        target = item["target"].to(device, dtype=torch.float)

        out = model(input_ids, attention_mask, token_type_ids)
        out = torch.sigmoid(out)

        y_true.append(target.to("cpu").numpy().reshape(-1,))
        y_pred_probab.append(out.to("cpu").numpy().reshape(-1,))

y_true_test = np.array(y_true).astype(int)
y_pred_probab_test = np.array(y_pred_probab)


In [ ]:
y_pred_test = np.where(y_pred_probab_test > .5, 1, 0)

eval.evaluate(y_true=y_true_test, y_pred=y_pred_test)

Save predictions

In [ ]:
pred_df = test[[ORIGINAL_TEXT, TARGET]].copy()
pred_df[PREDICTION] = y_pred_test.reshape(-1,)
save_to = os.path.join(data.reports_path, "roberta-prediction.pkl")
pred_df.to_pickle(save_to)

Evaluate embeddings

Extract embeddings for test reviews from fine-tuned model

In [ ]:
# Reduce to test set
df_features = df_features[df_features[SPLIT].isin([TEST])].copy()
df_features.reset_index(drop=True, inplace=True)

dataset = CustomDataset(
    df_features,
    MODEL_NAME,
    max_length=MAX_TOKEN_COUNT
)
dataset_loader = DataLoader(dataset, shuffle=False, batch_size=1, num_workers=NUM_WORKERS)

y_true = []
reviews = []
embeddings = []
with torch.no_grad():
    for i, item in enumerate(dataset_loader):
        reviews.append(item["text"])
        input_ids = item["input_ids"].to(device, dtype=torch.long)
        attention_mask = item["attention_mask"].to(device, dtype=torch.long)
        token_type_ids = item["token_type_ids"].to(device, dtype=torch.long)
        target = item["target"].to(device, dtype=torch.float)

        out = model.l1(input_ids, attention_mask, token_type_ids)
        embeddings.append(out.pooler_output.to("cpu").numpy().reshape(-1,))
        y_true.append(target.to("cpu").numpy().reshape(-1,))

y_true = np.array(y_true).astype(int)
reviews = ["<br>".join((textwrap.wrap(x[0]))) for x in reviews]
embeddings = np.array(embeddings)

k-Means (Floyd)

In [ ]:
distortions = []
inertias = []
mapping1 = {}
mapping2 = {}
K = range(1, 5)

for k in tqdm(K):
    # Building and fitting the model
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeanModel = kmeans.fit(embeddings)
    distortions.append(
        sum(np.min(
            cdist(embeddings, kmeanModel.cluster_centers_, 'euclidean'),
            axis=1
        )) / embeddings.shape[0]
    )
    inertias.append(kmeanModel.inertia_)
    mapping1[k] = sum(np.min(
        cdist(embeddings, kmeanModel.cluster_centers_, 'euclidean'),
        axis=1
    )) / embeddings.shape[0]
    mapping2[k] = kmeanModel.inertia_

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12,6), constrained_layout=True)
ax[0].plot(K, distortions, 'bx-')
ax[0].set_xlabel('k')
ax[0].set_ylabel('Distortion')
ax[0].set_title('Elbow Method (Distortion)')
ax[1].plot(K, inertias, 'bx-')
ax[1].set_xlabel('k')
ax[1].set_ylabel('Inertia')
ax[1].set_title('Elbow Method (Inertia)')
fig.suptitle("k-Means (Floyd)");

Elbow method suggests $k=2$

In [ ]:
# Useful constants
EMBEDDINGS = ["llm_{i}" for i in range(model.l1.config.hidden_size)]
CLUSTER = "cluster"
sentiment_map_inverse = {config.SENTIMENT_MAP[c]: c for c in config.SENTIMENT_MAP}

# Utility functions to compute purity of clusters
def compute_purity(df):
    purity_scores = {}
    clusters_unique = df[CLUSTER].unique()
    for c in clusters_unique:
        labels = df[df[CLUSTER]==c][TARGET].tolist()
        sorted_labels = sorted(list(set(labels)), key=labels.count, reverse=True)
        mode = sorted_labels[0]
        score = sum([1 if y==mode else 0 for y in labels]) / len(labels)
        mode_label = sentiment_map_inverse[mode]
        purity_scores[c] = (score, mode_label)
    return purity_scores

In [ ]:
k = 2
clustering_method = KMeans(n_clusters=k, random_state=0)
clusters = clustering_method.fit(embeddings)

embeddings_df = pd.DataFrame(embeddings, columns=EMBEDDINGS)
embeddings_df[ORIGINAL_TEXT] = reviews
embeddings_df[CLUSTER] = clusters.labels_
embeddings_df[TARGET] = y_true.reshape(-1,)

scores = compute_purity(embeddings_df)
scores

In [ ]:
dim_reduction_method = TSNE(n_components=2, initialization="pca", learning_rate="auto")
embeddings_reduced = dim_reduction_method.fit_transform(embeddings)

embeddings_df["x"] = embeddings_reduced[:,0]
embeddings_df["y"] = embeddings_reduced[:,1]

In [ ]:
fig = px.scatter(
    embeddings_df,
    y="y",
    x="x",
    color=CLUSTER,
    symbol=,
    hover_data=[TARGET, ORIGINAL_TEXT],
    title="k-Means"
)
fig.update_layout(uniformtext_mode="hide")
fig.update_layout(legend_orientation="h")
fig.show()